# 🏦 AI-Driven Loan Risk Analysis Pipeline
### Objective: Predict loan default risk, segment applicants, and recommend actions.
**Dataset:** Kaggle Loan Default Prediction
**Architecture:** Medallion (Bronze -> Silver -> Gold)
**Tech:** PySpark, MLflow, Databricks SQL

In [0]:
%pip install kaggle mlflow

import os
# Configure Kaggle (Ensure kaggle.json is uploaded to /Workspace/kaggle/kaggle.json)
os.environ['KAGGLE_CONFIG_DIR'] = "/Workspace/kaggle"
os.chmod("/Workspace/kaggle/kaggle.json", 0o600)

# Create Catalog and Schemas
spark.sql("CREATE CATALOG IF NOT EXISTS loan_risk_ai")
spark.sql("CREATE SCHEMA IF NOT EXISTS loan_risk_ai.bronze")
spark.sql("CREATE SCHEMA IF NOT EXISTS loan_risk_ai.silver")
spark.sql("CREATE SCHEMA IF NOT EXISTS loan_risk_ai.gold")
spark.sql("CREATE SCHEMA IF NOT EXISTS loan_risk_ai.ml")

print("✅ Environment Configured & Schemas Created")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 149.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.4/597.4 kB 27.7 MB/s eta 0:00:00
  Attempting uninstall: blinker
    Found existing installation: blinker 1.7.0
    Not uninstalling blinker at /usr/lib/python3/dist-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-234cb8dd-b078-458a-96ff-fccae6ae9f06
    Can't uninstall 'blinker'. No files were found to uninstall.
  Attempting uninstall: mlflow-skinny
    Found existing installation: mlflow-skinny 2.22.0
    Not uninstalling mlflow-skinny at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.epheme

## 1. Bronze Layer – Raw Data Ingestion
Downloading the Loan Default dataset from Kaggle and saving it as a raw Delta table.

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS loan_risk_ai.bronze.raw_files;


In [0]:
%sh
cd /Volumes/loan_risk_ai/bronze/raw_files

kaggle datasets download -d nikhil1e9/loan-default
unzip -o loan-default.zip
ls -lh


Dataset URL: https://www.kaggle.com/datasets/nikhil1e9/loan-default
License(s): CC0-1.0


100%|██████████| 7.86M/7.86M [00:00<00:00, 17.2MB/s]



Archive:  loan-default.zip
  inflating: Loan_default.csv        
total 32M
-rwxrwxrwx 1 spark-234cb8dd-b078-458a-96ff-fc nogroup  24M Jan 29 17:22 Loan_default.csv
-rwxrwxrwx 1 spark-234cb8dd-b078-458a-96ff-fc nogroup 7.9M Jan 29 17:22 loan-default.zip


In [0]:
raw_df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/Volumes/loan_risk_ai/bronze/raw_files/Loan_default.csv")
)

raw_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("loan_risk_ai.bronze.loan_raw")

print(f"✅ Bronze Table Created with {raw_df.count()} rows")
display(raw_df.limit(5))

✅ Bronze Table Created with 255347 rows


LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


## 2. Silver Layer – Data Cleaning & Validation
Handling nulls, fixing data types, and filtering invalid records.

In [0]:
from pyspark.sql.functions import col, when, count
from pyspark.sql.types import DoubleType, IntegerType

# Load Bronze
bronze_df = spark.table("loan_risk_ai.bronze.loan_raw")

# 1. Rename columns for clarity (optional, but recommended)
silver_df = bronze_df.withColumnRenamed("LoanID", "loan_id") \
    .withColumnRenamed("Age", "applicant_age") \
    .withColumnRenamed("Income", "annual_income") \
    .withColumnRenamed("LoanAmount", "loan_amount") \
    .withColumnRenamed("CreditScore", "credit_score") \
    .withColumnRenamed("MonthsEmployed", "months_employed") \
    .withColumnRenamed("NumCreditLines", "num_credit_lines") \
    .withColumnRenamed("InterestRate", "interest_rate") \
    .withColumnRenamed("LoanTerm", "loan_term") \
    .withColumnRenamed("DTIRatio", "dti_ratio") \
    .withColumnRenamed("Education", "education") \
    .withColumnRenamed("EmploymentType", "employment_type") \
    .withColumnRenamed("MaritalStatus", "marital_status") \
    .withColumnRenamed("HasMortgage", "has_mortgage") \
    .withColumnRenamed("HasDependents", "has_dependents") \
    .withColumnRenamed("LoanPurpose", "loan_purpose") \
    .withColumnRenamed("HasCoSigner", "has_cosigner") \
    .withColumnRenamed("Default", "label") 

# 2. Drop duplicates
silver_df = silver_df.dropDuplicates(["loan_id"])

# 3. Fill Numeric Nulls with Median
numeric_cols = ["applicant_age", "annual_income", "credit_score", "dti_ratio"]
for c in numeric_cols:
    median_val = silver_df.approxQuantile(c, [0.5], 0.01)[0]
    silver_df = silver_df.fillna({c: median_val})

# 4. Fill Categorical Nulls
silver_df = silver_df.fillna({
    "education": "Unknown",
    "employment_type": "Unemployed",
    "marital_status": "Single"
})

# Save to Silver
silver_df.write.format("delta").mode("overwrite").saveAsTable("loan_risk_ai.silver.loan_clean")
display(silver_df.limit(5))

loan_id,applicant_age,annual_income,loan_amount,credit_score,months_employed,num_credit_lines,interest_rate,loan_term,dti_ratio,education,employment_type,marital_status,has_mortgage,has_dependents,loan_purpose,has_cosigner,label
I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


## 3. Gold Layer – Feature Engineering
Creating risk buckets, encoding categories, and preparing vectors for ML.

In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, Imputer
from pyspark.ml import Pipeline

# Load Silver
silver_df = spark.table("loan_risk_ai.silver.loan_clean")

# 1. Create Business Logic Features
gold_df = silver_df.withColumn(
    "CreditRiskSegment",
    when(col("credit_score") > 750, "Excellent")
    .when(col("credit_score") > 650, "Good")
    .when(col("credit_score") > 550, "Fair")
    .otherwise("Poor")
).withColumn(
    "IncomeToLoanRatio",
    col("annual_income") / col("loan_amount")
)

# 2. Encode Categorical Columns
cat_cols = ["education", "employment_type", "marital_status", "loan_purpose", "CreditRiskSegment"]
indexers = [
    StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep") 
    for c in cat_cols
]

# 3. Assemble Features
feature_cols = [
    "applicant_age", "annual_income", "loan_amount", "credit_score", 
    "months_employed", "num_credit_lines", "interest_rate", "loan_term", 
    "dti_ratio", "IncomeToLoanRatio",
    "education_idx", "employment_type_idx", "marital_status_idx", "loan_purpose_idx", "CreditRiskSegment_idx"
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features", handleInvalid="skip")

# Pipeline execution
pipeline = Pipeline(stages=indexers + [assembler])
model_prep = pipeline.fit(gold_df)
gold_ready = model_prep.transform(gold_df)

# Save Gold Table
gold_ready.select("features", *gold_df.columns) \
    .write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("loan_risk_ai.gold.loan_features")

print("✅ Gold Features Table Created")

✅ Gold Features Table Created


## 4. Model Training & MLflow Tracking
Training a Gradient Boosted Tree Classifier to predict default risk.

In [0]:
%sql
-- Ensure the ML schema exists
CREATE SCHEMA IF NOT EXISTS loan_risk_ai.ml;

-- Create the Volume specifically for ML artifacts
CREATE VOLUME IF NOT EXISTS loan_risk_ai.ml.mlflow_artifacts;

In [0]:
import mlflow
import mlflow.spark
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# 1. Define a writable Volume path for ML artifacts
ml_artifact_path = "/Volumes/loan_risk_ai/ml/mlflow_artifacts"
dbutils.fs.mkdirs(ml_artifact_path)

# Load Data
data = spark.table("loan_risk_ai.gold.loan_features")   
train_df, test_df = data.randomSplit([0.8, 0.2], seed=42)

# Set Experiment
mlflow.set_experiment("/Users/" + dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get() + "/loan_risk_prediction")

with mlflow.start_run(run_name="GBT_Loan_Default"):
    # Initialize Model
    gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=20, maxDepth=5)
    
    # Train
    model = gbt.fit(train_df)
    
    # Evaluate
    predictions = model.transform(test_df)
    evaluator = BinaryClassificationEvaluator(labelCol="label")
    auc = evaluator.evaluate(predictions)
    
    # Log Metrics & Model
    mlflow.log_metric("auc", auc)
    
    # 🔥 FIX: Point dfs_tmpdir to the UC Volume path
    mlflow.spark.log_model(
        spark_model=model, 
        artifact_path="model", 
        dfs_tmpdir=ml_artifact_path
    )
    
    print(f"🚀 Model Trained Successfully! ROC-AUC: {auc:.4f}")

2026/01/29 17:31:18 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/29 17:31:21 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-234cb8dd-b078-458a-96ff-fc/tmpo6w8j_tt/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/01/29 17:31:21 INFO mlflow.models.model: Model logged without a signature. Signatures are required for Databricks UC model registry as they validate model inputs and denote the expected schema of model outputs. Please set `input_example` parameter when l

🚀 Model Trained Successfully! ROC-AUC: 0.7286


## 5. AI Innovation – Scoring & Recommendations
Applying the model to generate risk probabilities and suggested actions.

In [0]:
from pyspark.sql.functions import udf, lit
from pyspark.sql.types import DoubleType

# Define UDF to extract probability
get_prob = udf(lambda v: float(v[1]), DoubleType())

# Apply Model to Full Dataset (Scoring)
scored_df = model.transform(data)

# Extract Probability & Define Rules
final_scored = scored_df.withColumn("default_probability", get_prob(col("probability"))) \
    .withColumn(
        "RiskLevel",
        when(col("default_probability") > 0.7, "Critical Risk")
        .when(col("default_probability") > 0.4, "Moderate Risk")
        .otherwise("Low Risk")
    ).withColumn(
        "RecommendedAction",
        when(col("RiskLevel") == "Critical Risk", "Reject or Require Co-Signer")
        .when(col("RiskLevel") == "Moderate Risk", "Request Additional Documentation")
        .otherwise("Auto-Approve")
    )

# Save Final Recommendations
final_scored.select(
    "loan_id", "applicant_age", "credit_score", "annual_income", 
    "loan_amount", "default_probability", "RiskLevel", "RecommendedAction"
).write.format("delta").mode("overwrite").saveAsTable("loan_risk_ai.gold.loan_recommendations")

display(spark.table("loan_risk_ai.gold.loan_recommendations").limit(10))

loan_id,applicant_age,credit_score,annual_income,loan_amount,default_probability,RiskLevel,RecommendedAction
MZH5E8WTE8,40,350,85946,20737,0.10527940859190399,Low Risk,Auto-Approve
BJN03E1VGP,66,570,120424,55533,0.07441739435240013,Low Risk,Auto-Approve
XAP7JW48HR,20,335,58288,175916,0.15573706216178684,Low Risk,Auto-Approve
1KXJ882E41,19,452,113081,157873,0.0993589011907684,Low Risk,Auto-Approve
ZIFRLS8QJB,43,336,104821,98662,0.06910943542254955,Low Risk,Auto-Approve
HM8GL6LSIP,41,475,43416,113352,0.1406959908032155,Low Risk,Auto-Approve
X6BLC4F1XB,69,570,65827,235592,0.07384421955242215,Low Risk,Auto-Approve
GO0NOS0T9J,55,678,146831,112094,0.09986569309226145,Low Risk,Auto-Approve
G5MQ1UUEAX,36,511,71099,76022,0.07239894424562177,Low Risk,Auto-Approve
R6LP1CTEJO,40,548,22690,31288,0.07979883611057792,Low Risk,Auto-Approve


## 6. Analytics & Insights
SQL Queries for the business dashboard.

In [0]:
%sql
-- 1. Distribution of Risk Levels
SELECT RiskLevel, count(*) as ApplicantCount, round(avg(credit_score),0) as AvgCreditScore
FROM loan_risk_ai.gold.loan_recommendations
GROUP BY RiskLevel
ORDER BY ApplicantCount DESC;

RiskLevel,ApplicantCount,AvgCreditScore
Low Risk,249370,575.0
Moderate Risk,5646,557.0
Critical Risk,331,528.0


In [0]:
%sql
-- 2. High-Value Moderate Risk Applicants (Opportunity for Manual Review)
SELECT loan_id, annual_income, loan_amount, default_probability, RecommendedAction
FROM loan_risk_ai.gold.loan_recommendations
WHERE RiskLevel = 'Moderate Risk' AND annual_income > 80000
ORDER BY default_probability ASC
LIMIT 10;

loan_id,annual_income,loan_amount,default_probability,RecommendedAction
RK3K7EWHS9,93195,109688,0.4002443752799203,Request Additional Documentation
TEMKZK8F29,113190,230890,0.4005878225290841,Request Additional Documentation
OWF95G1XZT,129470,87073,0.40061855387565715,Request Additional Documentation
6LJ9HMUY33,138561,105158,0.40120871116280676,Request Additional Documentation
BZZZVABLHJ,126321,243716,0.40127824015336966,Request Additional Documentation
LEI218MTLQ,103749,148124,0.40136869630479177,Request Additional Documentation
8N46SM4ACA,138474,102096,0.40165161651123915,Request Additional Documentation
SHLDMCU96X,87820,201303,0.4017060230609162,Request Additional Documentation
16ZQB8E00Z,129765,201342,0.40171297810036766,Request Additional Documentation
55RPSL9N3Z,97774,214536,0.40174421920783476,Request Additional Documentation


In [0]:
%sql
SHOW TABLES IN loan_risk_ai.gold;

database,tableName,isTemporary
gold,loan_features,false
gold,loan_recommendations,false
